# Physics Informed Neural Networks <br> F1 Car Front Wing Aerodymanics

## PINN

In [ ]:
import numpy as np
import pandas as pd
import torch
import plotly.express as px
import plotly.graph_objects as go
from pinn import PINN
import os

In [ ]:
data_dir = "/Users/ggito/repos/pinns/data"
model_dir = os.path.join(data_dir, "models")
points_file_name = "front_wing_points_final.csv"
model_name = "v3"
model_checkpoint_num = 87

In [ ]:
wing_df = pd.read_csv(os.path.join(data_dir, points_file_name))

In [ ]:
device = torch.device("cpu")

In [ ]:
x_min, y_min, z_min = wing_df.min()
x_max, y_max, z_max = wing_df.max()

overall_min = min(x_min, y_min, z_min)
overall_max = max(x_max, y_max, z_max)

xyz_range = (overall_min, overall_max)

In [ ]:
def show_wing():
  df = px.data.iris()
  fig = px.scatter_3d(wing_df, x='x', y='y', z='z')
  fig.update_traces(marker_size = 1)
  fig.update_traces(marker_color = 'slategrey')
  fig.update_layout(
      scene = dict(
          xaxis = dict(range=xyz_range),
          yaxis = dict(range=xyz_range),
          zaxis = dict(range=xyz_range)))

  fig.show()

In [ ]:
# load model
input_dim = 4
output_dim = 4
hidden_units = [1024, 1024, 1024]
pinn = PINN(input_dim, output_dim, hidden_units, model_name=None).to(device)
pinn, _ = pinn.load_checkpoint_num(optimizer=None, checkpoint_dir=model_dir, model_name=model_name, checkpoint_num=model_checkpoint_num, mode='test')
pinn.eval()

In [ ]:
# testing domain
x_max = 1
y_max = 1
z_max = 1
t_max = 2 # TODO: set t>2 to test outside of training domain

Nx = 30
Ny = 30
Nz = 30
Nt = 50

dx = x_max / (Nx - 1)
dy = y_max / (Ny - 1)
dz = z_max / (Nz - 1)
dt = t_max / (Nt - 1)

x_test = np.linspace(0, x_max, Nx)
y_test = np.linspace(0, y_max, Ny)
z_test = np.linspace(0, z_max, Nz)
t_test = np.linspace(0, t_max, Nt)

x_grid, y_grid, z_grid = np.meshgrid(x_test, y_test, z_test, indexing='ij')

In [ ]:
# testing points
xyz_combinations = torch.cartesian_prod(torch.tensor(x_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.float32), torch.tensor(z_test, dtype=torch.float32))
num_points = xyz_combinations.shape[0]

In [ ]:
# model outputs of each time step will be stored in these lists
# for example, u_test[0] will contain u for all grid points at t = 0, u_test[-1] will contain u for all grid points at t = t_max
u_test = []
v_test = []
w_test = []
p_test = []
velocity_norm = []

In [ ]:
def predict():
  # iterate over time and store the model outputs
  for _t in t_test:
    _xyzt_combinations = torch.cat((xyz_combinations, torch.full((num_points, 1), _t, dtype=torch.float32)), dim=1)
    _input_xyzt = _xyzt_combinations.to(device)
    _output_uvwp = pinn(_input_xyzt)

    _u_test = _output_uvwp[:, 0].to("cpu").detach().numpy()
    _v_test = _output_uvwp[:, 1].to("cpu").detach().numpy()
    _w_test = _output_uvwp[:, 2].to("cpu").detach().numpy()
    _p_test = _output_uvwp[:, 3].to("cpu").detach().numpy()
    _velocity_norm = np.linalg.norm([_u_test, _v_test, _w_test], axis=0)
    
    u_test.append(_u_test)
    v_test.append(_v_test)
    w_test.append(_w_test)
    p_test.append(_p_test)
    velocity_norm.append(_velocity_norm)

In [ ]:
predict()

In [ ]:
# min & max values of the model outputs

velocity_norm = np.array(velocity_norm)
min_velocity_norm = velocity_norm.min()
max_velocity_norm = velocity_norm.max()

min_pressure = np.array(p_test).min()
max_pressure = np.array(p_test).max()

In [ ]:
def test_velocity():
  frames = []

  for t_frame in range(Nt):
    _scatter = go.Scatter3d(
        x=wing_df['x'],
        y=wing_df['y'],
        z=wing_df['z'],
        mode='markers',
        marker=dict(size=1, color='slategrey')
    )

    _cone = go.Cone(
        x=x_grid.flatten(),
        y=y_grid.flatten(),
        z=z_grid.flatten(),
        u=u_test[t_frame],
        v=v_test[t_frame],
        w=w_test[t_frame],
        colorscale='jet',
        cmin=min_velocity_norm,
        cmax=max_velocity_norm,
        colorbar_title='Velocity<br>Magnitude',
        opacity=0.2,
        sizemode="scaled",
        sizeref=max_velocity_norm
    )

    _frame = go.Frame(data=[_scatter, _cone])

    frames.append(_frame)

  # Create figure
  fig = go.Figure(
      data=[go.Scatter3d(),
            go.Cone()])
      
  fig.update_layout(
          scene = dict(
            xaxis=dict(range=[0, x_max], autorange=False),
            yaxis=dict(range=[0, y_max], autorange=False),
            zaxis=dict(range=[0, z_max], autorange=False),
          ))

  fig.update(frames=frames)

  fig.update_layout(updatemenus=[dict(type="buttons",
                            buttons=[dict(label="Play",
                                          method="animate",
                                          args=[None, dict(frame=dict(redraw=True,fromcurrent=True, mode='immediate'))])])])

  fig.show()

In [ ]:
def test_pressure():

  frames = []

  for t_frame in range(Nt):
    _scatter = go.Scatter3d(
        x=wing_df['x'],
        y=wing_df['y'],
        z=wing_df['z'],
        mode='markers',
        marker=dict(size=1, color='slategrey')
    )

    _scatter_p = go.Scatter3d(
        x=x_grid.flatten(),
        y=y_grid.flatten(),
        z=z_grid.flatten(),
        mode='markers',
        marker=dict(
          size=4,  # Adjust the size of the marker if needed
          color=p_test[t_frame],  # Set the color of the markers based on the pressure values
          colorscale='jet',  # Choose a colorscale (e.g., Viridis, Jet, etc.)
          cmin=min_pressure,
          cmax=max_pressure,
          colorbar=dict(title='Pressure'),  # Optional colorbar to indicate the scale
          opacity=0.1  # Adjust the opacity if needed
        )
    )

    _frame = go.Frame(data=[_scatter, _scatter_p])

    frames.append(_frame)

  # Create figure
  fig = go.Figure(
      data=[go.Scatter3d(),
            go.Scatter3d()])
      
  fig.update_layout(
          scene = dict(
            xaxis=dict(range=[0, x_max], autorange=False),
            yaxis=dict(range=[0, y_max], autorange=False),
            zaxis=dict(range=[0, z_max], autorange=False),
          ))

  fig.update(frames=frames)

  fig.update_layout(updatemenus=[dict(type="buttons",
                            buttons=[dict(label="Play",
                                          method="animate",
                                          args=[None, dict(frame=dict(redraw=True,fromcurrent=True, mode='immediate'))])])])

  fig.show()

In [ ]:
test_velocity()

In [ ]:
test_pressure()